In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')
import json
import numpy as np

In [2]:

from src.utils.DatabaseConnect import CacheConnector_02_04_25

DB_LINK = "mongodb://192.168.43.146:27020/"

connector = CacheConnector_02_04_25(DB_LINK)
connector.train_02_04_25.insert_one({"order" : 0, "stuff" : 987})
connector.train_02_04_25.insert_one({"order" : 1, "stuff" : 870987})
connector.train_02_04_25.find_one({"order" : 1})

{'_id': ObjectId('67ffe4fcc341bf3203b3ced4'), 'order': 1, 'stuff': 870987}